In [ ]:
# default_exp tabular.preprocessing

In [ ]:
#hide
from nbdev.showdoc import *

# Preprocessor for tabular models

In [ ]:
#export
from fastai.tabular.all import *
from fastai.data.all import *
from fastai.vision.data import get_grid
from enveco.data.las import *
from enveco.data.image import *
import matplotlib.patches as mpl_patches
from typing import Tuple
from fastai.metrics import *

## Preprocessing

Preprocess csv-files of our field plot data into `TabularPandas` to feed into models.

In [ ]:
#export

def get_lidar_feature(row, path, feature_func, min_h:float=1.5, mask_plot=True):
    "Function for LiDAR opening and processing steps"
    las_data = las_to_df(f'{path}/{row.sampleplotid}.las')
    if mask_plot == True: las_data = mask_plot_from_lidar(las_data, row.x, row.y)
    features = feature_func(las_data, min_h)
    return features

In [ ]:
#export

def get_image_procs(row, path, radius=31, mask_plot=True):
    "Function for tif opening and processing steps"
    image_data = open_geotiff(f'{path}/{row.sampleplotid}.tif')
    if mask_plot == True: image_data = mask_plot_from_image(image_data, radius=radius)
    metrics = calc_image_metrics(image_data)
    return metrics

In [ ]:
# export

from tqdm import tqdm
tqdm.pandas()

class EnvecoPreprocessor():
    
    def __init__(self, train_path, valid_path, test_path, **kwargs):
        self.train_df = pd.read_csv(train_path)
        self.train_df = self.train_df.rename(columns = lambda x: re.sub('[\.]+', '_', x))
        self.valid_df = pd.read_csv(valid_path)
        self.valid_df = self.valid_df.rename(columns = lambda x: re.sub('[\.]+', '_', x))
        self.test_df = pd.read_csv(test_path)
        self.test_df = self.test_df.rename(columns = lambda x: re.sub('[\.]+', '_', x))
        self.train_df['is_valid'] = 0
        self.valid_df['is_valid'] = 1
        self.train_val_df = pd.concat((self.train_df, self.valid_df))
        
        
    def preprocess_lidar(self, target_col, path, min_h:float=1.5, mask_plot:bool=True, height_features:bool=True,
                         point_features:bool=True, intensity_features:bool=True, height_quantiles:bool=True,
                         point_proportions:bool=True, canopy_densities:bool=True, normalize:bool=True,
                         log_y:bool=False) -> Tuple[TabularPandas, TabularPandas]:
        "Preprocess data and return (train_val, test) -tuple. Optionally log-transform target column with np.log1p"
        trainval = self.train_val_df.copy()
        test = self.test_df.copy()
        feature_cols = point_cloud_metric_cols
        trainval[point_cloud_metric_cols] = trainval.progress_apply(lambda row: point_cloud_metrics(f'{path}/{row.sampleplotid}.las',
                                                                                                    row.x, row.y, 
                                                                                                    min_h, mask_plot), 
                                                                    axis=1, result_type='expand')
        test[point_cloud_metric_cols] = test.progress_apply(lambda row: point_cloud_metrics(f'{path}/{row.sampleplotid}.las',
                                                                                            row.x, row.y, 
                                                                                            min_h, mask_plot), 
                                                            axis=1, result_type='expand')
        procs = None
        if normalize:
            procs = [Normalize]#.from_stats(*norm_stats)]
        trainval_tb = TabularPandas(trainval, procs=procs,
                                    cont_names=feature_cols, y_names=target_col,
                                    splits=ColSplitter(col='is_valid')(trainval))
        test_tb = TabularPandas(test, procs=procs,
                                cont_names=feature_cols, y_names=target_col)
        return trainval_tb, test_tb
    
    def preprocess_image(self, target_col, path, radius:int=31, mask_plot=True) -> Tuple[TabularPandas, TabularPandas]:
        "Preprocess dataframes and return (train_val, test) -tuple"
        # TODO
        pass
    
    def preprocess(self, target_col, path, lidar:bool=True, image:bool=True, min_h:float=1.5, radius:int=31, 
                   mask_plot:bool=True) -> Tuple[TabularPandas, TabularPandas]:
        "Preprocess dataframes and return (train_val, test) -tuple"
        # TODO
        pass

In [ ]:
# hide

from nbdev.export import notebook2script
notebook2script()
!nbdev_build_docs

Converted 00_data.image.ipynb.
Converted 01_data.las.ipynb.
Converted 02_tabular.preprocessing.ipynb.
Converted 03_model.inception3dv3.ipynb.
Converted 04_interpretation.ipynb.
Converted 05_metrics.ipynb.
Converted 06_model.ensemble.ipynb.
Converted index.ipynb.
converting: /mnt/d/Users/E1005164/enveco/nbs/02_tabular.preprocessing.ipynb
converting /mnt/d/Users/E1005164/enveco/nbs/index.ipynb to README.md
